<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/rag_ppc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q "pypdf" "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "PyPDF2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc.pdf"

--2024-04-03 14:39:52--  https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico
Resolving www.uffs.edu.br (www.uffs.edu.br)... 200.135.49.107
Connecting to www.uffs.edu.br (www.uffs.edu.br)|200.135.49.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1493834 (1.4M) [application/pdf]
Saving to: ‘data/ppc.pdf’

data/ppc.pdf        100%[===================>]   1.42M  1.21MB/s    in 1.2s    

2024-04-03 14:39:54 (1.21 MB/s) - ‘data/ppc.pdf’ saved [1493834/1493834]



In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/ppc.pdf")
pages = loader.load_and_split()

In [ ]:
pages[0]

Document(page_content='MINISTÉRIO DA EDUCAÇÃO\nUNIVERSIDADE FEDERAL DA FRONTEIRA SUL – UFFS\nPRÓ-REITORIA DE GRADUAÇÃO\nDIRETORIA DE PEDAGÓGICA\nPROJETO PEDAGÓGICO DO CURSO DE GRADUAÇÃO EM\nCIÊNCIA DA COMPUTAÇÃO – BACHARELADO\nChapecó-SC, setembro  de 2017.\nProjeto Pedagógico do Curso de Graduação em Ciência da Computação – Bacharelado 1', metadata={'source': 'data/ppc.pdf', 'page': 0})

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

documents = text_splitter.transform_documents(pages)

In [ ]:
len(documents)

492

In [ ]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(documents, embedder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
query = "Qual a ementa de geometria analítica?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 5)

for page in docs:
  print(page.page_content)

CAROLI,  C.;  CALLIOLI,  C.  A.;  FEITOSA,  M.  O.  Matrizes,  vetores  e  Geometria
Analítica. São Paulo: Nobel, 1984.
REIS, G. L. Dos; SILV A, V . V . Geometria Analítica . Rio de Janeiro: LTC, 1996.
SANTOS, F. J. dos; FERREIRA, S. F. Geometria Analítica.  Porto Alegre: Artmed, 2009.
Projeto Pedagógico do Curso de Graduação em Ciência da Computação – Bacharelado 57
York: John Wiley & Sons, 1967. v. 2.
LARSON, R.; HOSTETLER, R. P.; EDWARDS, B. H. Cálculo. 8. ed. São Paulo: McGraw
Hill, 2006. 1 v.
______. Cálculo. 8 ed. São Paulo: McGraw Hill, 2006. 2 v.
SALAS, H. E. Cálculo. 9. ed. Rio de Janeiro: LTC, 2005. 1 v.
______. Cálculo. 9. ed. Rio de Janeiro: LTC, 2005. 2 v.
SIMMONS, G. F. Cálculo com geometria analítica. São Paulo: McGraw Hill, 1987. 1 v.
______. Cálculo com geometria analítica. São Paulo: McGraw Hill, 1987. 2 v.
Projeto Pedagógico do Curso de Graduação em Ciência da Computação – Bacharelado 68
MINISTÉRIO DA EDUCAÇÃO
UNIVERSIDADE FEDERAL DA FRONTEIRA SUL – UFFS
PRÓ-REITORIA

In [ ]:
!pip install huggingface-hub -q

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import torch
# import transformers
# from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
# from peft import PeftModel, PeftConfig

# model_id = 'recogna-nlp/bode-7b-alpaca-pt-br'

# config = PeftConfig.from_pretrained(model_id)


# model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True, return_dict=True, load_in_8bit=True, device_map='auto')

# model.eval()



import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.1,
    max_new_tokens=256
)

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

In [ ]:
qa_with_sources_chain({"query" : "Qual a ementa de algebra linear?"})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Qual a ementa de algebra linear?',
 'result': "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nMINISTÉRIO DA EDUCAÇÃO\nUNIVERSIDADE FEDERAL DA FRONTEIRA SUL – UFFS\nPRÓ-REITORIA DE GRADUAÇÃO\nDIRETORIA DE PEDAGÓGICA\nCódigo COMPONENTE CURRICULAR CréditosHoras\nGEX607 ÁLGEBRA LINEAR 460\nEMENTA\nMatrizes. Determinantes. Sistemas de equações lineares. Espaços vetoriais. Transformações\nlineares. Espaços com produto interno. Autovalores e autovetores. Diagonalização.\nOBJETIVO\nResolver sistemas de equações lineares utilizando operações elementares. Calcular a inversa\nde matrizes usando operações elementares. Compreender os conceitos de transformação\nlinear, autovalor, autovetor e ter contato com aplicações da Álgebra Linear em problemas\nreais.\nREFERÊNCIAS BÁSICAS\nANTON, H.; RORRES, C. Álgebra linear com aplicações . 8. ed. Porto Alegre: Bookman,\n2001.\nBO

In [ ]:
import os
import PyPDF2
from PyPDF2 import PdfReader

# Importing necessary modules from langchain and langchain_openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.callbacks import get_openai_callback
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough


def chunk_processing(pdf):
    """
    Process a PDF file, extracting text and splitting it into chunks.
    """
    pdf_reader = PdfReader(pdf)

    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text=text)
    return chunks

def embeddings(chunks):
    """
    Create embeddings for text chunks using OpenAI.
    """
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_KEY)
    # Create vector store using FAISS
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    return vector_store
